In [1]:
### See notebooks 'France_IRIS_revenue_by_zip' and 'France_INSEE_number_of_household_with_children_by_zip_and_dep' 
### for construction of revenue and number of households with children  csv

#### Here we will load the number of client by zip and compute pentration by zip

In [1]:
import os
import pandas as pd
import numpy as np
from datetime import date, datetime, timedelta
today = date.today().strftime("%Y%m%d")

In [2]:
import gc

### Preliminar functions and imports

In [3]:
def customer_category_attribution(x):
    if x==1:
        return 'One-timer'
    if x==2:
        return 'Two-timer'
    else:
        return 'Recurring'

In [4]:
def flatten (table):
    if type(table.columns)==pd.MultiIndex:
        columns_to_look = [name_tmp for name_tmp in table.columns]

        columns_df = [ str(t[0])+'_'+str(t[1]) for t in columns_to_look]
        columns_df.insert(0,table.index.name)

        df = pd.DataFrame(columns = columns_df)

        index = 0
        for i in table.index:
            row = [table[r][i] for r in columns_to_look]
            row.insert(0,i)
            df.loc[index] = row
            index = index + 1
        return(df)
    else :
        table = pd.DataFrame(table)
        table.reset_index(level=0, inplace=True)
        return table

In [5]:
df_articles = pd.read_csv('./Vertbaudet/France/src/Original/Articles/Export_FR_Articles_Julie.csv',
                                     usecols = ["DEPARTEMENT_NIV2","CODSAI","REFSTK", "TOP_MN_Max"])

df_codsoc = pd.read_csv('./Vertbaudet/France/src/20210921_CODSOC.csv')

df_client_dept = pd.read_csv('./20210923_Export_tab_FR/20210921_INDIVIDUS.tab', sep = '\t',
                 usecols = ['IDIND', 'CODPOST', "DATPREMCDEWEB"])

clients_mag = pd.read_csv("./Vertbaudet/France/src/20211004_UNIQUE_IDIND_TICKET.csv",sep=",")

df_client = pd.read_csv('./Vertbaudet/France/Output_old/iris_med_rev_clients.csv', usecols = ['IDIND', 'Mediane'])

df_IRIS_by_dep = pd.read_excel('./Vertbaudet/France/INSEE_data/INSEE_IRIS_by_dep.xlsx', usecols = ['CODGEO','MED18'])

C:\Users\pierrick\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


# Import

In [ ]:
gc.collect()
df = pd.DataFrame()
for year in range(2017,2021):
    print(year)
    gc.collect()
    df_tmp = pd.read_csv('./Vertbaudet/France/src/Original/Lignes_commandes/Export_FR_LIGNECOMMANDE_'+str(year)+'.csv', 
                              encoding = 'unicode_escape', 
                              usecols = ['IDIND', 'IDFOYER', 'DN', 'NUMCDE', 'DBI'])

    ### Merge with article table to get DEPARTEMENT_NIV2
    #df_tmp = df_tmp.merge(df_articles, how = 'left', on = ["CODSAI","REFSTK"])
    
    ### Replace $null$
    df_tmp = df_tmp.replace({'$null$': np.nan})
    ### Clean IDIND index 
    df_tmp.IDIND = df_tmp.IDIND.astype(float)
    df_tmp.IDFOYER = df_tmp.IDFOYER.astype(float)
    
    ### Filter mag clients
    #df_tmp = df_tmp[~df_tmp.IDIND.isin(clients_mag.IDIND)]

    ### Filter non-french command with CODSOC
    df_tmp = df_tmp.merge(df_codsoc, how = 'left', on = 'NUMCDE')
    df_tmp = df_tmp[df_tmp.CODSOC==0]

    #### Merge to get Mediane rev
    df_tmp = df_tmp.merge(df_client, how = 'left', on = 'IDIND')

    ### add departments and cohort
    df_tmp = df_tmp.merge(df_client_dept.replace({'$null$': np.nan}), how = 'left', on = 'IDIND')
    df_tmp = df_tmp[df_tmp.DATPREMCDEWEB != '$null$']
    df_tmp['DEPARTEMENT'] = df_tmp['CODPOST'].apply(lambda x : str(x)[:2])
    df_tmp['COHORT'] = pd.to_datetime(df_tmp["DATPREMCDEWEB"]).dt.year
    
    ### filter cohort
    #df_tmp = df_tmp[(df_tmp['COHORT'] >= 2017) & (df_tmp['COHORT'] < 2021)]
    
    ### filter on DN <= DB
    #df_tmp = df_tmp[df_tmp.DN <= df_tmp.DBI]
    #df_tmp = df_tmp[df_tmp.DN >= 0]

    df = pd.concat([df,df_tmp])
  
    del df_tmp

### Average median rev by department

In [ ]:
df_dep = df.drop_duplicates('IDFOYER').groupby('DEPARTEMENT').agg({'Mediane': 'mean'})

df_dep =  df_dep.merge(df_IRIS_by_dep, how = 'left', left_index = True, right_on = 'CODGEO')

df_dep = df_dep.rename(columns = {'Mediane': 'MEDIAN_VB', 'MED18': 'MEDIAN_INSEE'})

df_dep['MED_VB_VS_AVERAGE'] = df_dep['MEDIAN_VB']/df_dep['MEDIAN_INSEE']

df_dep.to_excel('./Vertbaudet/France/Output/med_vb_vs_med_insee_by_dep.xlsx')

In [39]:
df.drop_duplicates('IDFOYER')['Mediane'].mean()

22991.06170094055

### DN by CODPOST

In [8]:
df.groupby('CODPOST').agg({'DN':sum}).to_excel('./Vertbaudet/France/Output/DN_by_zip.xlsx')

### OT/TT/Recurring categories by Departments

In [12]:
df['MONTH'] = pd.to_datetime(df.DATPREMCDEWEB).dt.month

In [13]:
### Add CLIENT_CATEGORY column according to number of commands
df_nb_commande = df.groupby('IDIND').agg({'NUMCDE': pd.Series.nunique})
df_nb_commande.columns = ['NB_ORDERS']
df = pd.merge(df,df_nb_commande,on='IDIND',how='left',suffixes=(False,False))
df['CLIENT_CATEGORY'] = df.NB_ORDERS.apply(lambda x: customer_category_attribution(x))
gc.collect()

75

In [17]:
### Create output df
df_output_cat = df.groupby(['DEPARTEMENT','CLIENT_CATEGORY']).agg({'DN': sum, 'IDIND': pd.Series.nunique})
df_output_cat = df_output_cat.unstack()

### Drop non-valid dept numbers
df_output_cat = df_output_cat[df_output_cat['DN','Recurring'].notna()].drop(index = ['00', '20','97','98'])

In [18]:
### Get statistics for the whole country
for crit in ['DN', 'IDIND']:
    for cat in ['One-timer', 'Two-timer', 'Recurring']:
        print('% of '+crit+ ' of '+cat+' :')
        print( (df_output_cat[crit,cat].sum()) / (df_output_cat[crit].sum().sum()))

% of DN of One-timer :
0.15552989210407267
% of DN of Two-timer :
0.11069566407799897
% of DN of Recurring :
0.7337744438179283
% of IDIND of One-timer :
0.5442781794858389
% of IDIND of Two-timer :
0.17560564972521955
% of IDIND of Recurring :
0.2801161707889415


In [21]:
df_output_cat.to_excel('./Vertbaudet/France/Output/category_by_dep.xlsx')

### Number of clients by univ

In [ ]:
df_nb_client_univ = df.drop_duplicates(subset = ['IDIND', 'DEPARTEMENT_NIV2']).groupby(['DEPARTEMENT', 'DEPARTEMENT_NIV2']).agg({'IDIND':'count', 'DEPARTEMENT_NIV2':'first'})

In [ ]:
df_nb_clients_by_dep = df.groupby('DEPARTEMENT').agg({'IDIND': pd.Series.nunique})

df_nb_client_univ = df_nb_client_univ.merge(df_nb_clients_by_dep, how = 'left', on = 'DEPARTEMENT')

df_nb_client_univ = df_nb_client_univ.rename(columns = {'IDIND_x': 'COUNT_CLIENT_UNIV', 'IDIND_y': 'COUNT_CLIENT_TOTAL'})

df_nb_client_univ['PENETRATION'] = df_nb_client_univ['COUNT_CLIENT_UNIV']/df_nb_client_univ['COUNT_CLIENT_TOTAL']

df_nb_client_univ['COUNT_CLIENT_UNIV'].sum()/df_nb_client_univ['COUNT_CLIENT_TOTAL'].sum()

# df_nb_client_univ.groupby(['DEPARTEMENT', 'DEPARTEMENT_NIV2']).agg({'COUNT_CLIENT_UNIV':'first', 'COUNT_CLIENT_TOTAL':'first', 'PENETRATION':'first'}).unstack().to_excel('./Vertbaudet/France/Output/'+today+'_penetration_univ_by_dept_17_20.xlsx')

df_count = df_nb_client_univ.groupby(['DEPARTEMENT', 'DEPARTEMENT_NIV2']).agg({'COUNT_CLIENT_UNIV':'first', 'COUNT_CLIENT_TOTAL':'first', 'PENETRATION':'first'}).unstack()

for univ in ['BEBE', 'CHAMBRE_ET_LITERIE', 'CHAU', 'DECO', 'FM', 'JOUETS',
       'KIDS_AUTRE', 'NON_PAP_ENFANT', 'PAP_FILLE', 'PAP_GARCON', 'PUERI',
       'TEXTILE_HOME']:
    print('Average penetration of '+univ+' in France: ')
    print(df_count['COUNT_CLIENT_UNIV',univ].sum()/df_count['COUNT_CLIENT_TOTAL',univ].sum())

df_ubd = df_ubd[df_ubd.notna()]

In [46]:
df_ubd = df.groupby(['DEPARTEMENT', 'DEPARTEMENT_NIV2']).agg({'DN':sum}).unstack()

In [ ]:
df_ubd.to_excel('./Vertbaudet/France/Output/univ_by_dep_year_17_20.xlsx')

### LTV by dep

In [ ]:
import datetime
from datetime import datetime, time ,date ,timedelta

# Add DAT_12 and DAT_14 column
df['DAT_12'] = df['DATPREMCDEWEB'] + timedelta(days=365)
df['DAT_24'] = df['DATPREMCDEWEB'] + timedelta(days=730)

### Add LTV_12 column
df_ltv_12 = df[df['DATCDE']<=df['DAT_12']][['IDIND', 'DN']]
df_ltv_12 = df_ltv_12.groupby('IDIND').agg({'DN': sum})
df_ltv_12 = df_ltv_12.rename(columns = {'DN': 'LTV_12'})
print(df_ltv_12['LTV_12'].mean())
df = df.merge(df_ltv_12, how = 'left', on = 'IDIND')

del df_ltv_12

### Add LTV_24 column
df_ltv_24 = df[df['DATCDE']<=df['DAT_24']][['IDIND', 'DN']]
df_ltv_24 = df_ltv_24.groupby('IDIND').agg({'DN': sum})
df_ltv_24 = df_ltv_24.rename(columns = {'DN': 'LTV_24'})
print(df_ltv_24['LTV_24'].mean())
df = df.merge(df_ltv_24, how = 'left', on = 'IDIND')

del df_ltv_24

gc.collect()

In [ ]:
df_LTV_12_by_dept = df[df['SENIORITY']>365].drop_duplicates('IDIND').groupby('DEPARTEMENT').agg({'LTV_12': 'mean'})
df_LTV_12_by_dept.to_excel('./Vertbaudet/France/Output/'+today+'_ltv_12_by_dep.xlsx')